# Optimal portfolios with short-sales constraints

In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from scipy.optimize import minimize
from cvxopt import matrix
from cvxopt.solvers import qp as Solver, options as SolverOptions

### Inputs

In [2]:
##### Inputs
# Risk-free rate
RF = 0.01

# Expected returns
MNS = np.array([0.06, 0.065, 0.08])

# Standard deviations
SDS = np.array([0.15, 0.165, 0.21])

# Correlations
C  = np.identity(3)
C[0, 1] = C[1, 0] = 0.75
C[0, 2] = C[2, 0] = 0.75
C[1, 2] = C[2, 1] = 0.75

# Covariance matrix
COV = np.diag(SDS) @ C @ np.diag(SDS)

## Efficient Frontier

In [3]:
def frontier(means, cov, target, Shorts):
    n = len(means)
    Q = matrix(cov, tc="d")
    p = matrix(np.zeros(n), (n, 1), tc="d")
    if Shorts==1:
        # Constraint: short-sales allowed
        G = matrix(np.zeros((n,n)), tc="d")
    else:
        # Constraint: short-sales not allowed
        G = matrix(-np.identity(n), tc="d")
    h = matrix(np.zeros(n), (n, 1), tc="d")        
    # Fully-invested constraint + E[r] = target
    A = matrix(np.vstack((np.ones(n), means)), (2, n), tc="d")
    b = matrix([1, target], (2, 1), tc="d")
    sol = Solver(Q, p, G, h, A, b)
    wgts = np.array(sol["x"]).flatten() if sol["status"] == "optimal" else np.array(n * [np.nan])
    return wgts
SolverOptions['show_progress'] = False

Efficient frontier with short-selling

In [4]:
NUM_TARGETS = 50
df = pd.DataFrame(dtype='float',columns=['target_expret','w1','w2','w3','port_expret','port_sd'], index=np.arange(NUM_TARGETS))
df.target_expret = np.linspace(0.9*MNS.min(), 1.1*MNS.max(),NUM_TARGETS)
for i in df.index:
    wgts = frontier(MNS, COV, df.loc[i,'target_expret'], 1)
    df.loc[i,['w1','w2','w3']] = wgts.round(4)
    df.loc[i,'port_expret'] = wgts @ MNS
    df.loc[i,'port_sd'] = np.sqrt(wgts @ COV @ wgts)
df.head(10)

,target_expret,w1,w2,w3,port_expret,port_sd
0,0.054000,0.9723,0.4369,-0.4092,0.054000,0.146736
1,0.054694,0.9412,0.4322,-0.3734,0.054694,0.145856
2,0.055388,0.9100,0.4274,-0.3375,0.055388,0.145131
3,0.056082,0.8789,0.4227,-0.3016,0.056082,0.144560
4,0.056776,0.8478,0.4179,-0.2657,0.056776,0.144148
5,0.057469,0.8167,0.4131,-0.2298,0.057469,0.143895
6,0.058163,0.7856,0.4084,-0.1939,0.058163,0.143801
7,0.058857,0.7544,0.4036,-0.1580,0.058857,0.143868
8,0.059551,0.7233,0.3989,-0.1222,0.059551,0.144095
9,0.060245,0.6922,0.3941,-0.0863,0.060245,0.144482


Efficient frontier without short-selling

In [5]:
df_ssc = pd.DataFrame(dtype='float',columns=['target_expret','w1','w2','w3','port_expret','port_sd'], index=np.arange(NUM_TARGETS))
df_ssc.target_expret = np.linspace(MNS.min(), MNS.max(),NUM_TARGETS)
for i in df_ssc.index:
    wgts = frontier(MNS, COV, df_ssc.loc[i,'target_expret'], 0)
    df_ssc.loc[i,['w1','w2','w3']] = wgts.round(4)
    df_ssc.loc[i,'port_expret'] = wgts @ MNS
    df_ssc.loc[i,'port_sd'] = np.sqrt(wgts @ COV @ wgts)
df_ssc.head()

,target_expret,w1,w2,w3,port_expret,port_sd
0,0.060000,1.0000,-0.0000,0.0,0.060000,0.150000
1,0.060408,0.9184,0.0816,0.0,0.060408,0.148125
2,0.060816,0.8367,0.1633,0.0,0.060816,0.146800
3,0.061224,0.7551,0.2449,0.0,0.061224,0.146038
4,0.061633,0.6736,0.3264,0.0,0.061633,0.145850


## Global minimum variance

In [6]:
def gmv(means, cov, Shorts):
    n = len(means)
    Q = matrix(cov, tc="d")
    p = matrix(np.zeros(n), (n, 1), tc="d")
    if Shorts==1:
        # Constraint: short-sales allowed
        G = matrix(np.zeros((n,n)), tc="d")
    else:
        # Constraint: short-sales not allowed
        G = matrix(-np.identity(n), tc="d")
    h = matrix(np.zeros(n), (n, 1), tc="d")
    # Constraint: fully-invested portfolio
    A = matrix(np.ones(n), (1, n), tc="d")
    b = matrix([1], (1, 1), tc="d")
    sol = Solver(Q, p, G, h, A, b)
    wgts = np.array(sol["x"]).flatten() if sol["status"] == "optimal" else np.array(n * [np.nan])
    return wgts

GMV with shorting

In [7]:
wgts_gmv = gmv(MNS, COV, 1)
print(f'GMV weights\t\t {wgts_gmv.round(4)}')

# Portfolio expected return
gmv_expret = wgts_gmv @ MNS
print(f'GMV Expected Return:\t {gmv_expret: ,.4f}')

# Portfolio standard deviation
gmv_sd = np.sqrt(wgts_gmv @ COV @ wgts_gmv)
print(f'GMV Standard Deviation:\t {gmv_sd: ,.4f}')

GMV weights		 [ 0.783  0.408 -0.191]
GMV Expected Return:	  0.0582
GMV Standard Deviation:	  0.1438


GMV without shorting

In [8]:
wgts_gmv_ssc = gmv(MNS, COV, 0)
print(f'GMV w/o shorting weights\t\t {wgts_gmv_ssc.round(4)}')

# Portfolio expected return
gmv_ssc_expret = wgts_gmv_ssc @ MNS
print(f'GMV w/o shorting Expected Return:\t {gmv_ssc_expret: ,.4f}')

# Portfolio standard deviation
gmv_ssc_sd = np.sqrt(wgts_gmv_ssc @ COV @ wgts_gmv_ssc)
print(f'GMV w/o shorting Standard Deviation:\t {gmv_ssc_sd: ,.4f}')

GMV w/o shorting weights		 [0.6875 0.3125 0.    ]
GMV w/o shorting Expected Return:	  0.0616
GMV w/o shorting Standard Deviation:	  0.1458


## Tangency Portfolio

In [9]:
def tangency(means, cov, rf, Shorts):
    n = len(means)
    def f(w):
        mn = w @ means
        sd = np.sqrt(w @ cov @ w)
        return -(mn - rf) / sd
    # Initial guess (equal-weighted)
    w0 = (1/n)*np.ones(n)
    # Constraint: fully-invested portfolio
    A = np.ones(n)
    b = 1
    cons = [{"type": "eq", "fun": lambda x: A @ x - b}]
    if Shorts==1:
        # No short-sale constraint
        bnds = [(None, None) for i in range(n)] 
    else:
        # Short-sale constraint
        bnds = [(0, None) for i in range(n)] 
    # Optimization
    TOL = 10**(-10)
    wgts = minimize(f, w0, bounds=bnds, constraints=cons, options={'ftol':TOL}).x
    return wgts

Tangency with shorting

In [10]:
wgts_tang = tangency(MNS, COV, RF, 1)
print(f'Tangency weights\t\t {wgts_tang.round(4)}')

# Portfolio expected return
tang_expret = wgts_tang @ MNS
print(f'Tangency Expected Return:\t {tang_expret: ,.4f}')

# Portfolio standard deviation
tang_sd = np.sqrt(wgts_tang @ COV @ wgts_tang)
print(f'Tangency Standard Deviation:\t {tang_sd: ,.4f}')

Tangency weights		 [0.3812 0.3465 0.2723]
Tangency Expected Return:	  0.0672
Tangency Standard Deviation:	  0.1566


Tangency without shorting

In [11]:
wgts_tang_ssc = tangency(MNS, COV, RF, 0)
print(f'Tangency w/o shorting weights\t\t {wgts_tang.round(4)}')

# Portfolio expected return
tang_ssc_expret = wgts_tang_ssc @ MNS
print(f'Tangency w/o shorting Expected Return:\t\t {tang_ssc_expret: ,.4f}')

# Portfolio standard deviation
tang_ssc_sd = np.sqrt(wgts_tang_ssc @ COV @ wgts_tang_ssc)
print(f'Tangency w/o shorting Standard Deviation:\t {tang_ssc_sd: ,.4f}')

Tangency w/o shorting weights		 [0.3812 0.3465 0.2723]
Tangency w/o shorting Expected Return:		  0.0672
Tangency w/o shorting Standard Deviation:	  0.1566


### The full picture

In [12]:
fig = go.Figure()

# Plot underlying assets
trace1= go.Scatter(x=[SDS[0]], y=[MNS[0]],mode="markers", marker=dict(size=10, color="red"), name='Asset 1')
fig.add_trace(trace1)
trace2= go.Scatter(x=[SDS[1]], y=[MNS[1]],mode="markers", marker=dict(size=10, color="red"), name='Asset 2')
fig.add_trace(trace2)
trace3= go.Scatter(x=[SDS[2]], y=[MNS[2]],mode="markers", marker=dict(size=10, color="red"), name='Asset 3')
fig.add_trace(trace3)

# Plot frontier with shorting
string =  "Asset 1: %{customdata[0]:.1%}<br>"
string += "Asset 2: %{customdata[1]:.1%}<br>"
string += "Asset 3: %{customdata[2]:.1%}<br>"
string += "<extra></extra>"
trace= go.Scatter(x=df.port_sd, y=df.port_expret,mode="lines", marker=dict(color="black"),
    customdata=df[['w1','w2','w3']],hovertemplate=string, name='Frontier with shorting')
fig.add_trace(trace)

# Plot GMV with shorting
trace= go.Scatter(x=[gmv_sd], y=[gmv_expret] ,
    mode="markers", marker=dict(size=10, color="black"),
    customdata=wgts_gmv.reshape(1,3),hovertemplate=string, name='GMV with shorting')
fig.add_trace(trace)

# Plot frontier without shorting
trace= go.Scatter(x=df_ssc.port_sd, y=df_ssc.port_expret,mode="lines",marker=dict(color="blue"),
    customdata=df_ssc[['w1','w2','w3']],hovertemplate=string, name='Frontier without shorting')
fig.add_trace(trace)

# Plot GMV without shorting
trace= go.Scatter(x=[gmv_ssc_sd], y=[gmv_ssc_expret] ,
    mode="markers", marker=dict(size=10, color="blue"),
    customdata=wgts_gmv_ssc.reshape(1,3),hovertemplate=string, name='GMV without shorting')
fig.add_trace(trace)


# Formatting
fig.layout.yaxis["title"] = "Expected Return"
fig.layout.xaxis["title"] = "Standard Deviation"
fig.update_yaxes(tickformat=".1%")
fig.update_xaxes(tickformat=".1%")
fig.update_xaxes(range=[0.6 * df["port_sd"].min(), 1.1 * df["port_sd"].max()])
fig.update_yaxes(range=[0.7 * df["port_expret"].min(), 1.1 * df["port_expret"].max()])
fig.update_layout(legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01))
fig.show()